# ガウス波束

ガウス波束をあるポテンシャルに打ち込むときの時間発展をシミュレーションします。

まず各種定数とパラメタを定義しておきます。

In [1]:
const ħ=1.0;    #Dirac定数ℏ
const m=0.5;    #粒子の質量

const Δ=15.0;    #ガウス波束の運動量のバラつき
const k=100.0;    #初期条件における運動量の期待値
const x0=-0.3;    #初期条件におけるピークの位置

const a=-1.0;    #左端
const b= 1.0;    #右端
const dx=0.004;    #座標の刻み幅
const L=(Int)(1+(b-a)/dx);    #座標に関する配列のサイズ

const tL=801;    #時間に関する配列のサイズ
const dt=dx^2/4.0;   #時間の刻み幅
const t0=0.0;    #開始時刻
const t1=t0+dx*(tL-1);    #終了時刻

V0とV(x)を変更することで色々なポテンシャルの場合が試せます。<br />
とりあえず有限高さの階段型ポテンシャルの場合です。

In [2]:
V0=0.7*k^2/2m    #ポテンシャルの高さ

function V(x)    #階段型ポテンシャル
    if x<0
        return 0
    else
        return V0
    end
end

V (generic function with 1 method)

In [3]:
function VArray()    #プロット用
    r=Array{Float64}(L)
    
    for i in 1:L
        x=a+(i-1)*dx
        r[i]=V(x)/(k^2/2m)
    end
    
    return r
end

VArray (generic function with 1 method)

波動関数の初期条件を与えます。

In [4]:
function Ψ0()    #波動関数の初期条件
    Ψ=Array{Complex64}(L)
    
    for i in 1:L
        x=a+(i-1)*dx
        Ψ[i]=exp(-Δ^2*(x-x0)^2/ħ^2 + im*k*x/ħ)
    end
    
    return Ψ
end

Ψ0 (generic function with 1 method)

波動関数$\Psi$は、一定の幅$dx$ごとの値を保持した配列の形で表現しています。その$\Psi$に対して$\hat{H}\Psi$は以下のように計算できます。ただし、ハミルトニアンに現れる$x$の微分は差分で近似します。

In [5]:
function HΨ(Ψ)
    r=Array{Complex64}(L)    #p̂Ψ/2m+VΨを格納
    r[1]=r[L]=0.0    #両端は十分遠方とみなして0にする
    
    for i in 2:L-1
        x=a+(i-1)*dx
        r[i]=-ħ^2/2m*(Ψ[i+1]-2Ψ[i]+Ψ[i-1])/dx^2+V(x)*Ψ[i]
    end
    
    return r
end

HΨ (generic function with 1 method)

シュレーディンガー方程式を解いていきます。<br />
$\hat{H}\Psi$は上のように計算してあるため、$t$に関する一階の微分方程式として解けると考えられます。
ルンゲ・クッタ法を用いて以下のように$\frac{\Delta\Psi}{\Delta t}$を推定します。

In [6]:
function slope(Ψ)
    k1=-im.*HΨ(Ψ)./ħ
    k2=-im.*HΨ(Ψ.+0.5dt.*k1)./ħ
    k3=-im.*HΨ(Ψ.+0.5dt.*k2)./ħ
    k4=-im.*HΨ(Ψ.+dt.*k3)./ħ
    
    return (k1.+2k2.+2k3.+k4)./6.0
end

slope (generic function with 1 method)

推定した$\frac{\Delta\Psi}{\Delta t}$をもとに初期条件から時間発展させていきます。

In [7]:
Ψ=Array{Array{Complex64}}(tL)

Ψ[1]=Ψ0()
for i in 2:tL
    Ψ[i]=Ψ[i-1].+slope(Ψ[i-1])*dt
end

あとは得られた結果をアニメーションとして出力するだけです。<br />
少し時間がかかります。

In [8]:
using Plots

In [9]:
gr()
ENV["PLOTS_TEST"]="true"

"true"

In [10]:
anim=@animate for t in 1:tL
    plot(a:dx:b,[abs.(Ψ[t]).^2,VArray()], 
        leg=false,
        tickfont=(9,"Consolas"),
        xlims=(a,b),
        ylims=(-0.1,1.2))
end

Plots.Animation("/tmp/tmpmpe8dd", String["000001.png", "000002.png", "000003.png", "000004.png", "000005.png", "000006.png", "000007.png", "000008.png", "000009.png", "000010.png"  …  "000792.png", "000793.png", "000794.png", "000795.png", "000796.png", "000797.png", "000798.png", "000799.png", "000800.png", "000801.png"])

In [11]:
gif(anim,"./anim.gif",fps=30)

INFO: Saved animation to /mnt/d/Documents/julia/IJulia/anim.gif


Plots.AnimatedGif("/mnt/d/Documents/julia/IJulia/anim.gif")